# Dataset Preprocessing Sweep
Trying to set up a new template for a notebook that will run a simple 5-fold cross-validation XGBoost model on a variety of dataset permutations.

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True
libraries = ['xgboost', 'lightgbm', 'catboost', 'widedeep-SAINT']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"dataset_sweep_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
#     !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

#     !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if USE_GPU: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if USE_GPU:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
import optuna
import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.preprocessing import StandardScaler MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from boruta import BorutaPy
from BorutaShap import BorutaShap
import category_encoders as ce

In [6]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/oct2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    altdatapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/')
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [7]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Ex-Model Config

In [8]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

## Data Loading

In [15]:
# loading the full training data set to get the feature correlations to target, for K-means clustering later
df = pd.read_feather(datapath/'train.feather')
# df_corr = df.corr() # getting the correlations of the features
# corr_target = df_corr.loc['target':'target'] # pulling out just the correlation of features with the target, as a 1-row df (for Series, it'd be df_corr.loc['target'])
corr_target = load(altdatapath/'corr_target.joblib')

In [16]:
corr_target.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
target,0.004067,-0.029324,-0.015663,0.036279,0.019811,-0.012301,-0.012332,0.013528,-0.043557,-0.002662,...,-0.00329,-0.003869,-0.004477,-0.004503,-0.004319,-0.004587,-0.002426,-0.005901,-0.0037,1.0


In [26]:
corr_target_x = corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
corr_target_abs = abs(corr_target_x) # just interested in magnitudes here
corr_sorted = corr_target_abs.sort_values(by='target', axis=1, ascending=False) # df columns of useful values by correlation with target, will be modified later
y = df.target # pulling out the dependent variable
X = df.drop('target', axis=1) # isolating the independent variables
del df # cleaning up memory
categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars

NameError: name 'df' is not defined

## Parameters

In [ ]:
# optuna 20211004, thru 106 trials on unaltered original dataset
params = {
    'n_estimators': 3878,
    'max_depth': 4,
    'learning_rate': 0.024785857161974977,
    'reg_alpha': 26.867682044658245,
    'reg_lambda': 10.839759074147148,
    'subsample': 0.8208581489835881,
    'min_child_weight': 8.829122644339664,
    'colsample_bytree': 0.906420714280384,
    'gamma': 1.472322916021486
}

In [26]:
def cross_validate_model(X, y, params:dict=best_xgboost_params, start_fold=0, 
                         exmodel_config=exmodel_config, wandb_config=wandb_config, 
                         random_state=42):
    """
    Modified version of cross-validation function from `stacking_manual_*.ipynb` series
    """
    # prepare for k-fold cross-validation; random-state here is notebook-wide, not per-model
    # shuffle on the initial sets, but not subsequently -- performing the same operation twice means a very different dataset
    kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=False)
        
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
        
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
#         print(f"type(train_ids) = {type(train_ids)} and train_ids.shape = {train_ids.shape}")
#         print(f"type(valid_ids) = {type(valid_ids)} and train_ids.shape = {valid_ids.shape}")
        if fold < start_fold: # skip folds that are already trained
            continue
        else:
            print(f"FOLD {fold}")
            print("---------------------------------------------------")
            
            
            y_train, y_valid = y[train_ids], y[valid_ids] # y will be an np.ndarray already; handling will be same regardless of model
            if isinstance(X, np.ndarray):
                X_train, X_valid = X[train_ids], X[valid_ids]
            else:
                X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for ce
            
            # exmodel_config['feature_count'] = len(X.columns)
#                 wandb.log({
#                     'feature_count': X_train.shape[1],
#                     'instance_count': X_train.shape[0],
#                     'encoder': str(encoder)
#                 })
#                 exmodel_config['instance_count'] = X_train.shape[0]
#                 exmodel_config['encoder'] = str(encoder)
#                     X_test = encoder.transform(X_test)
#                 y_train, y_valid = y[train_ids], y[valid_ids]

        # define models
        model = XGBClassifier(
            booster='gbtree',
            tree_method='gpu_hist',
            random_state=random_state,
            n_jobs=-1, 
            verbosity=1, 
            objective='binary:logistic',
            **params)
        if wandb_tracked:
            model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
        else:
            model.fit(X_train, y_train)
        y_valid_preds = model.predict_proba(X_valid)[:,1]

        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)

        # add the fold's predictions to the model's test-set predictions (will divide later)
        test_preds += model.predict_proba(X_test)[:,1]



#         valid_loss = log_loss(y_valid, y_pred)
        # give the valid AUC score, for edification
        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
        dump(model, Path(runpath/f"{library}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for {library} model is {model_valid_auc}")
    
    # finalize test preds
#     test_preds /= exmodel_config['kfolds']
    
    # save OOF preds and test-set preds
#     dump(oof_preds, Path(predpath/f"{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds_rs{random_state}_oof_preds.joblib"))
#     dump(test_preds, Path(predpath/f"{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds_rs{random_state}_test_preds.joblib"))
#     dump(oof_y, predpath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib")
    
    return model_valid_auc#, test_preds
        

In [28]:
b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
type(b)

numpy.ndarray

In [31]:
bdf = pd.DataFrame(b, index=X.index).join(y)
bdf.head()

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,target
0,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.200924,0.713191,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1
1,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.231828,0.457150,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1
2,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.247791,0.631949,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1
3,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.287755,0.455777,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.265285,0.503776,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1


(Following cells generate the correlations for the different feature selections, so that the process need not be repeated each iteration.)

In [36]:
# b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
# bdf = pd.DataFrame(b, index=X.index).join(y)
# bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered_green_corr_sorted.joblib']

In [37]:
# del b, bdf, bdf_corr_target, bdf_corr_target_x, bdf_corr_target_abs, bdf_corr_sorted

In [38]:
# b = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather')
# bdf = b.join(y)
# # bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib')

['/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_shap_200trials_corr_sorted.joblib']

In [14]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
    

#     dump(pca60, edapath/'PCA_60.joblibg')

    # use the original 286-feature dataset, or the 136-feature BorutaShap selected one
#     dataset = trial.suggest_categorical('dataset', ['X_orig.feather', 'X_boruta_shap_200trials.feather']) 
#     train_source = altdatapath/'X_orig.feather'
    # train_source = altdatapath/'train-WITH-KMeans_12cluster_kmeans++_maxiter1000_rs42.feather' #'X_boruta_shap_200trials.feather'
#     X = pd.read_feather(path=train_source)
#     y = load(datapath/'y.joblib')
    
#     # decides whether binary-encoded categoricals are encoded or not
#     cardinality_min = trial.suggest_categorical('cardinality_min', [0, 2]) 
        
    encoder_name = trial.suggest_categorical('encoder_name', ['woe', 'catboost', 'james-stein', 'loo', 'mestimate', 'target', 'hashing', None])
    if encoder_name:
        encode_before_kmeans = trial.suggest_categorical('encode_before_kmeans', [True, False]) # determines order

    # define dict of encoders, with names as keys and implementations as values
    encoders = {
        'woe': ce.WOEEncoder(cols=categoricals),
        'catboost': ce.CatBoostEncoder(cols=categoricals),
        'james-stein': ce.JamesSteinEncoder(cols=categoricals),
        'loo': ce.LeaveOneOutEncoder(cols=categoricals),
        'mestimate': ce.MEstimateEncoder(cols=categoricals),
        'target': ce.TargetEncoder(cols=categoricals),
        'hashing': ce.HashingEncoder(cols=categoricals),
    }
    
    # feature selection setup -- applied before preprocessing
    feature_selection = trial.suggest_categorical('feature_selection', ['BorutaShap', 'Boruta', None])
    k_means_method = trial.suggest_categorical('k_means_method', [25, 50, 100, 'k-means++', None]) # K-Means initialization method

    # now, switch datasets if feature selection is implemented; regardless, prepare appropriate K-Means setup (to be implemented later, in folds)
    if feature_selection: # create a subset of features if appropriate
        if feature_selection == 'BorutaShap':
            X = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather', index=X.index) # :: pd.DataFrame
            categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
            # k-means cluster feature generation setup 
            if k_means_method:
                corr_sorted = load(altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib') # load prepared correlations
                useful_features = list(corr_sorted.columns[:k_means_clusters])
        elif feature_selection == 'Boruta':
            X = pd.DataFrame(load(altdatapath/'X_boruta_200iter_filtered_green.joblib'), index=X.index)
            if k_means_method:
                corr_sorted = load(altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib') # load prepared correlations
                useful_features = list(corr_sorted.columns[:k_means_clusters])
            categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
    else:
        if k_means_method:
            k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
            useful_features = list(corr_sorted.columns[:k_means_clusters])
    
    # PCA dimensionality reduction setup -- applied at end of preprocessing
    pca_components = trial.suggest_categorical('pca_components', [50, 75, 'mle', None, 'NO'])
    
    # define k-fold splitter
    kfold = KFold(n_splits=5, shuffle=False)
        
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
            
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
        print(f"FOLD {fold}")
        print("---------------------------------------------------")
        
        y_train, y_valid = y[train_ids], y[valid_ids] # slicing syntax works on both pandas.Series and numpy.ndarray
        # category_encoders expects pandas.DataFrames
        X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for slicing
        
        # now, apply preprocessing
        if encoder_name: # if categorical encoding to be applied to high cardinality (2<x<100,000) categoricals...
            if k_means_method: # if k-means proceeding
                if encode_before_kmeans: # do category encoding, then clustering
                    # category encoding for high-cardinality categoricals
                    encoder = encoders[encoder_name]
                    X_train = encoder.fit_transform(X_train, y_train)
                    X_valid = encoder.transform(X_valid)

                    # k-means cluster feature generation
                    cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                    if k_means_method == 'k-means++':
                        kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                    else:
                        kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                    # fit on the training set only
                    X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                    X_valid_clusters = kmeans.transform(X_valid[useful_features])
                    # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
                    X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                    X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                    # join the cluster-distance features to the training and validation sets
                    X_train = X_train.join(X_train_clusters)
                    X_valid = X_valid.join(X_valid_clusters)

                else: # do k-means clustering, then do category encoding
                    cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                    if k_means_init_method == 'k-means++':
                        kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                    else:
                        kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_init_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                    X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                    X_valid_clusters = kmeans.transform(X_valid[useful_features])
                    X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                    X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                    X_train = X_train.join(X_train_clusters)
                    X_valid = X_valid.join(X_valid_clusters)

                    encoder = encoders[encoder_name]
                    X_train = encoder.fit_transform(X_train, y_train)
                    X_valid = encoder.transform(X_valid)
            
            else: # category encoding, but no k-means
                encoder = encoders[encoder_name]
                X_train = encoder.fit_transform(X_train, y_train)
                X_valid = encoder.transform(X_valid)
                
        else: # no category encoding
            if k_means_method: # if still doing k-means
                # k-means cluster feature generation
                cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                if k_means_method == 'k-means++':
                    kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                else:
                    kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                # fit on the training set only
                X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                X_valid_clusters = kmeans.transform(X_valid[useful_features])
                # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
                X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                # join the cluster-distance features to the training and validation sets
                X_train = X_train.join(X_train_clusters)
                X_valid = X_valid.join(X_valid_clusters)
            
        
        # now, PCA dimensionality reduction
        if pca_components != 'NO':
            pca = PCA(n_components=pca_components, random_state=42)
            X_train = pca.fit_transform(X_train)
            X_valid = pca.transform(X_valid)
            
        # define models
        model = XGBClassifier(
            booster='gbtree',
            tree_method='gpu_hist',
            random_state=random_state,
            n_jobs=-1, 
            verbosity=1, 
            objective='binary:logistic',
            **params)
        model.fit(X_train, y_train)
        y_valid_preds = model.predict_proba(X_valid)[:,1]

        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)

        # add the fold's predictions to the model's test-set predictions (will divide later)
        test_preds += model.predict_proba(X_test)[:,1]

        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
#         dump(model, Path(runpath/f"{library}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for is {model_valid_auc}")
    
    return valid_auc

In [15]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-15-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='dataset_20211025')
# setting n_jobs=1 bc wandb apparently requires this


[I 2021-10-21 15:04:29,479] A new study created in memory with name: CatBoost_lv2_20211021


In [17]:
for x in range(100):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())
    dump(study, filename=datapath/f'optuna_dataset_study_trial{x}_20211026.joblib')

[I 2021-10-21 15:07:44,984] Trial 0 finished with value: 0.8424986224444081 and parameters: {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4}. Best is trial 0 with value: 0.8424986224444081.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'iterations': 12487, 'depth': 10, 'learning_rate': 0.09454306819536169, 'random_strength': 60, 'od_wait': 329, 'reg_lambda': 12.60762738286178, 'border_count': 63, 'min_child_samples': 18, 'leaf_estimation_iterations': 4, 'value': 0.8424986224444081}.


ROC AUC Score of CatBoost = 0.8424986224444081


[I 2021-10-21 15:10:03,533] Trial 1 finished with value: 0.8535448089511177 and parameters: {'iterations': 21826, 'depth': 3, 'learning_rate': 0.41472250004816347, 'random_strength': 84, 'od_wait': 440, 'reg_lambda': 14.364097770082843, 'border_count': 91, 'min_child_samples': 7, 'leaf_estimation_iterations': 3}. Best is trial 1 with value: 0.8535448089511177.
wandb: WARNING Step must only increase in log calls.  Step 1 < 2; dropping {'iterations': 21826, 'depth': 3, 'learning_rate': 0.41472250004816347, 'random_strength': 84, 'od_wait': 440, 'reg_lambda': 14.364097770082843, 'border_count': 91, 'min_child_samples': 7, 'leaf_estimation_iterations': 3, 'value': 0.8535448089511177}.


ROC AUC Score of CatBoost = 0.8535448089511177


[I 2021-10-21 15:11:43,086] Trial 2 finished with value: 0.855637886238501 and parameters: {'iterations': 14094, 'depth': 5, 'learning_rate': 0.044809759182149515, 'random_strength': 14, 'od_wait': 598, 'reg_lambda': 26.912605343971038, 'border_count': 153, 'min_child_samples': 16, 'leaf_estimation_iterations': 1}. Best is trial 2 with value: 0.855637886238501.
wandb: WARNING Step must only increase in log calls.  Step 2 < 3; dropping {'iterations': 14094, 'depth': 5, 'learning_rate': 0.044809759182149515, 'random_strength': 14, 'od_wait': 598, 'reg_lambda': 26.912605343971038, 'border_count': 153, 'min_child_samples': 16, 'leaf_estimation_iterations': 1, 'value': 0.855637886238501}.


...

ROC AUC Score of CatBoost = 0.8566811885481668


[I 2021-10-21 17:27:24,242] Trial 98 finished with value: 0.8566895874215903 and parameters: {'iterations': 5230, 'depth': 7, 'learning_rate': 0.0020058986656578833, 'random_strength': 0, 'od_wait': 183, 'reg_lambda': 44.258478769990646, 'border_count': 107, 'min_child_samples': 7, 'leaf_estimation_iterations': 4}. Best is trial 88 with value: 0.8567014488546707.
wandb: WARNING Step must only increase in log calls.  Step 98 < 99; dropping {'iterations': 5230, 'depth': 7, 'learning_rate': 0.0020058986656578833, 'random_strength': 0, 'od_wait': 183, 'reg_lambda': 44.258478769990646, 'border_count': 107, 'min_child_samples': 7, 'leaf_estimation_iterations': 4, 'value': 0.8566895874215903}.


ROC AUC Score of CatBoost = 0.8566895874215903


[I 2021-10-21 17:28:39,373] Trial 99 finished with value: 0.8566584256346833 and parameters: {'iterations': 7963, 'depth': 7, 'learning_rate': 0.001908760024016461, 'random_strength': 11, 'od_wait': 208, 'reg_lambda': 45.519971812430654, 'border_count': 101, 'min_child_samples': 7, 'leaf_estimation_iterations': 4}. Best is trial 88 with value: 0.8567014488546707.
wandb: WARNING Step must only increase in log calls.  Step 99 < 100; dropping {'iterations': 7963, 'depth': 7, 'learning_rate': 0.001908760024016461, 'random_strength': 11, 'od_wait': 208, 'reg_lambda': 45.519971812430654, 'border_count': 101, 'min_child_samples': 7, 'leaf_estimation_iterations': 4, 'value': 0.8566584256346833}.


ROC AUC Score of CatBoost = 0.8566584256346833


In [20]:
study.best_trial.params

{'iterations': 2000,
 'depth': 6,
 'learning_rate': 0.002984126581340097,
 'random_strength': 0,
 'od_wait': 334,
 'reg_lambda': 33.469738674488084,
 'border_count': 158,
 'min_child_samples': 8,
 'leaf_estimation_iterations': 4}

In [21]:
wandb.log({'best_dataset_params': study.best_trial.params})
wandb.finish()

valid_AUC,▁▇▇██████████▆█▇████████▇███████████████
valid_AUC,0.85666
